<a href="https://colab.research.google.com/github/Gibbons-Lab/isb_course_2021/blob/main/16S_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦠 Amplicon Sequencing Data Analysis with Qiime 2

This notebook will accompany the first session of the 2021 ISB Virtual Microbiome Series. The presentation slides can be [found here](https://gibbons-lab.github.io/isb_course_2021/16S). 

Save your own local copy of this notebook by using `File > Save a copy in Drive`. At some point you may be promted to trust the notebook. We promise that it is safe 🤞

**Disclaimer:**

The Google Colab notebook environment will interpret any command as Python code by default. If we want to run bash commands we will have to prefix them by `!`. So any command you see with a leading `!` is a bash command and if you wanted to run it in your terminal you would omit the leading `!`. For example, if in the Colab notebook you ran `!wget` you would just run `wget` in your terminal. 

## Setup

QIIME 2 is usually installed by following the [official installation instructions](https://docs.qiime2.org/2021.8/install/). However, because we are using Google Colab and there are some caveats to using conda here, we will have to hack around the installation a little bit. But no worries, we provide a setup script below which does all this work for us. 😌 

So...let's start by pulling a local copy of the project repository down from GitHub.

In [ ]:
# !git clone https://github.com/gibbons-lab/isb_course_2021 materials
!git clone https://github.com/Bello-Bello/micro_biome_workpr materials

Cloning into 'materials'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 136 (delta 17), reused 0 (delta 0), pack-reused 76
Receiving objects: 100% (136/136), 188.58 MiB | 19.59 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Checking out files: 100% (94/94), done.


We will switch to working within the `materials` directory for the rest of the notebook.

In [ ]:
%cd materials

/content/materials


Now we are ready to set up our environment. This will take about 10-15 minutes. 

**Note**: This setup is only relevant for Google Colaboratory and will not work on your local machine. Please follow the [official installation instructions](https://docs.qiime2.org/2021.8/install/) for that.

In [ ]:
%run setup_qiime2

[16:33:58] 🐍 Miniconda is already installed. Skipped.                                              ]8;id=18567;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=100143;file:///content/materials/setup_qiime2#99\99]8;;\

           🔍 Qiime 2 is already installed. Skipped.                                               ]8;id=696042;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=346204;file:///content/materials/setup_qiime2#126\126]8;;\

           📊 Checking that Qiime 2 command line works...                                           ]8;id=230470;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=419611;file:///content/materials/setup_qiime2#56\56]8;;\

[16:34:00] 📊 Qiime 2 command line looks good 🎉                                                    ]8;id=572262;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=269133;file:///content/materials/setup_qiime2#62\62]8;;\

           🧹 Cleaned up unneeded files.                                                            ]8;id=359838;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=409854;file:///content/materials/setup_qiime2#51\51]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍                                  ]8;id=696541;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=313522;file:///content/materials/setup_qiime2#159\159]8;;\

## Our first QIIME 2 command

Let's remember our workflow for today.

![our workflow](https://github.com/Gibbons-Lab/isb_course_2021/raw/main/docs/16S/assets/steps.png)

The first thing we have to do is to get the data into an artifact.
We can import the data with the `import` action from the tools. For that we have to give
QIIME 2 a *manifest* (list of raw files) and tell it what *type of data* we
are importing and what *type of artifact* we want. 

**QoL Tip:** QIIME 2 commands can get very long. To split them up over several lines we can use `\` which means "continue on the next line".

In [ ]:
# !qiime tools import \
#   --type 'SampleData[SequencesWithQuality]' \
#   --input-path data/manifest.tsv \
#   --output-path sequences.qza \
#   --input-format SingleEndFastqManifestPhred33V2

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path data/manifest_pair_read.tsv \
  --output-path pair_sequences.qza \
  --input-format PairedEndFastqManifestPhred33V2

Imported data/manifest_pair_read.tsv as PairedEndFastqManifestPhred33V2 to pair_sequences.qza


Since we have quality information for the sequencing reads, let's also generate
our first visualization to inspect sequence quality. 

---

Qiime 2 commands can become pretty long. Here are some pointers to remember the
structure of a command:

```
qiime plugin action --i-argument1 ... --o-argument2 ...
```

Argument types usually begin with a letter denoting their meaning:

- `--i-...` = input files
- `--o-...` = output files
- `--p-...` = parameters
- `--m-...` = metadata

---

In this case we will use the `summarize` action from the `demux` plugin with the previously generated artifact as input and output the resulting visualization to the `qualities.qzv` file.

In [ ]:
!qiime demux summarize --i-data pair_sequences.qza --o-visualization qualities.qzv

Saved Visualization to: qualities.qzv


You can view the plot by downloading the .qzv file and opening it using http://view.qiime2.org. To download the file click on the folder symbol to the left, open the `materials` folder, and choose download from the dot menu next to the `qualities.qzv` file.

🤔 What do you observe across the read? Where would you truncate the reads?

# Denoising amplicon sequence variants

We will now run the DADA2 plugin which will do 3 things:

1. filter and trim the reads
2. find the most likely set of unique sequences in the sample (ASVs)
3. remove chimeras
4. count the abundances of each ASV


Since this step takes a bit, let's start the process and use the time to
understand what is happening:

In [ ]:
# !qiime dada2 denoise-single \
#     --i-demultiplexed-seqs sequences.qza \
#     --p-trunc-len 140 \
#     --p-n-threads 2 \
#     --output-dir dada2 --verbose

In [ ]:
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs pair_sequences.qza \
    --p-trunc-len-f 0 \
    --p-trunc-len-r 0\
    --p-n-threads 2 \
    --output-dir dada2 --verbose

Usage: qiime dada2 denoise-paired 
           [OPTIONS]

  This method denoises paired-end sequences,
  dereplicates them, and filters chimeras.

Inputs:
  --i-demultiplexed-seqs ARTIFACT 
    SampleData[PairedEndSequencesWithQuality]
                         The paired-end
                         demultiplexed sequences
                         to be denoised.
                                        [required]
Parameters:
  --p-trunc-len-f INTEGER
                         Position at which
                         forward read sequences
                         should be truncated due
                         to decrease in quality.
                         This truncates the 3' end
                         of the of the input
                         sequences, which will be
                         the bases that were
                         sequenced in the last
                         cycles. Reads that are
                         shorter than this value
                      

If this step takes too long or fails, you can also copy the results from the treasure chest. **However, don't run the next cell if the previous cell completed successfully.**

In [ ]:
# obscure magic that will only copy if the previous command failed
# ![ -d dada2 ] || cp -r treasure_chest/dada2 .

Ok, this step ran, but we should also make sure it kind of worked. One good way to tell if the identified ASVs are representative of the sample is to see how many reads were maintained throughout the pipeline. Here, the most common issues and solutions are:

**Large fraction of reads is lost during merging (only paired-end)**

![read overlap](https://gibbons-lab.github.io/isb_course_2020/16S/assets/read_overlap.png)

In order to merge ASVs DADA2 uses an overlap of 12 bases between forward and reverse reads by default. Thus, your reads must allow for sufficient overlap *after* trimming. So if your amplified region is 450bp long and you have 2x250bp reads and you trim the last 30 bases of each read, truncating the length to 220bp, the total length of covered sequence is 2x220 = 440 which is shorter than 450bp so there will be no overlap. To solve this issue trim less of the reads or adjust the `--p-min-overlap` parameters to something lower (but not too low).

<br>

**Most of the reads are lost as chimeric**

![read overlap](https://gibbons-lab.github.io/isb_course_2020/16S/assets/chimera.png)

This is usualluy an experimental issue as chimeras are introduced during amplification. If you can adjust your PCR, try to run fewer cycles. Chimeras can also be introduced by incorrect merging. If your minimum overlap is too small ASVs may be merged randomly. Possible fixes are to increase the `--p-min-overlap` parameter or run the analysis on the forward reads only (in our empirical observations, chimeras are more likely to be introduced in the joined reads). *However, losing between 5-25% of your reads to chimeras is normal and does not require any adjustments.*

Our denoising stats are contained in an artifact. To convert it to a visualization we can use `qiime metadata tabulate`.

In [ ]:
!qiime metadata tabulate \
    --m-input-file dada2/denoising_stats.qza \
    --o-visualization dada2/denoising-stats.qzv

Saved Visualization to: dada2/denoising-stats.qzv


What proprotion of reads was retained throughout the entire pipeline? Look at the final number of used reads (non-chimeric). What do you observe when comparing those values between samples and how might that affect diversity metrics?

# Phylogeny and diversity

## Building a tree

We can build a phylogenetic tree for our sequences using the following command:

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada2/representative_sequences.qza \
    --output-dir tree

Usage: qiime phylogeny align-to-tree-mafft-fasttree 
           [OPTIONS]

  This pipeline will start by creating a
  sequence alignment using MAFFT, after which
  any alignment columns that are
  phylogenetically uninformative or ambiguously
  aligned will be removed (masked). The
  resulting masked alignment will be used to
  infer a phylogenetic tree and then
  subsequently rooted at its midpoint. Output
  files from each step of the pipeline will be
  saved. This includes both the unmasked and
  masked MAFFT alignment from q2-alignment
  methods, and both the rooted and unrooted
  phylogenies from q2-phylogeny methods.

Inputs:
  --i-sequences ARTIFACT FeatureData[Sequence]
                          The sequences to be
                          used for creating a
                          fasttree based rooted
                          phylogenetic tree.
                                        [required]
Parameters:
  --p-n-threads VALUE Int % Range(1, None) |
    Str % Choices('a

We can create a visualization for the tree using the [empress](https://github.com/biocore/empress) Qiime 2 plugin.

In [ ]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

Saved Visualization to: tree/empress.qzv


This looks nice but is not particularly informative as is. The main utility of our tree will be in complementing our diversity analyses. It will tell us which ASVs are more or less related to one another, which will allow us to calculate different kinds of ecological diversity metrics.

## Alpha and Beta Diversity

![sample sources](https://github.com/Gibbons-Lab/isb_course_2021/raw/main/docs/16S/assets/sample_sources.png)

One of our main goals will be to compare the microbial composition from different environments. Qiime 2 has "run-all" command for diversity analyses. This will

1. Subsampe our samples to the same total number of reads (Why?) 
2. Calculate alpha and beta diversity measures
3. Visualize PCoA projections

In [ ]:
# !qiime dada2 align-to-tree-mafft-fasttree \
#     --i-sequences dada2/table.qza \
#     --output-dir tree

# !qiime demux summarize --i-data dada2/table.qza --o-visualization table_table.qzv

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table dada2/table.qza \
    --i-phylogeny tree/rooted_tree.qza \
    --p-sampling-depth 240 \
    --m-metadata-file data/metadata.tsv \
    --output-dir diversity

Usage: qiime diversity core-metrics-phylogenetic 
           [OPTIONS]

  Applies a collection of diversity metrics
  (both phylogenetic and non-phylogenetic) to a
  feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table
                          containing the samples
                          over which diversity
                          metrics should be
                          computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree
    Phylogeny[Rooted]     containing tip
                          identifiers that
                          correspond to the
                          feature identifiers in
                          the table. This tree can
                          contain tip ids that are
                          not present in the
                          table, but all feature
                          ids in the table must be
                          present in this tree.
              

## Statistical analyses

Let's first have a look at alpha diversity. Can we see a difference in the per-sample diversity across environments? And between animal-asociated and free-living communities?

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization diversity/alpha_groups.qzv

Saved Visualization to: diversity/alpha_groups.qzv


Now, let's use beta diversity to see how different the samples are from one another. First download `diversity/weighted_unifrac_emperor.qzv` and take a look. Do samples separate based on the environment?

We can check whether that separation is 'significant' by using a PERMANOVA test.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file data/metadata.tsv \
    --p-formula "Responses" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

Saved Visualization to: diversity/permanova.qzv


# Taxonomy

We will use a Bayes classifier trained on the GreenGenes database which can be downloaded from https://docs.qiime2.org/2021.4/data-resources/.

In [ ]:
# ! wget https://data.qiime2.org/2022.2/common/silva-138-99-nb-classifier.qza
# ! wget wget\
#   -O "85_otu_taxonomy.txt" \
#   "https://data.qiime2.org/2021.4/tutorials/training-feature-classifiers/85_otu_taxonomy.txt"

In [ ]:
# # !wget https://data.qiime2.org/2021.8/common/gg-13-8-99-515-806-nb-classifier.qza
# !wget wget \
#   -O "85_otu_taxonomy.txt" \
#   "https://data.qiime2.org/2021.4/tutorials/training-feature-classifiers/85_otu_taxonomy.txt"

In [ ]:
! wget https://data.qiime2.org/2022.2/common/silva-138-99-nb-classifier.qza

--2022-09-22 16:35:49--  https://data.qiime2.org/2022.2/common/silva-138-99-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2022.2/common/silva-138-99-nb-classifier.qza [following]
--2022-09-22 16:35:50--  https://s3-us-west-2.amazonaws.com/qiime2-data/2022.2/common/silva-138-99-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.194.120
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.194.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 531637835 (507M) [binary/octet-stream]
Saving to: ‘silva-138-99-nb-classifier.qza.1’

silva-138-99-nb-cla 100%[===================>] 507.01M  39.3MB/s    in 14s     

2022-09-22 16:36:04 (35.9 MB/s) - ‘silva-138-99-nb-classifier.qza.1’ saved [531

In [ ]:
# !wget https://data.qiime2.org/2021.8/common/gg-13-8-99-515-806-nb-classifier.qza

In [ ]:
%ls

16S_microbiom_data_gd.ipynb     pair_sequences.qza
Copy_of_16S_solutions_gd.ipynb  qualities.qzv
dada2/                          README.md
data/                           setup_qiime2
data2/                          silva-138-99-nb-classifier.qza
diversity/                      silva-138-99-nb-classifier.qza.1
LICENSE                         tree/


In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-reads dada2/representative_sequences.qza \
    --i-classifier silva-138-99-nb-classifier.qza \
    --p-n-jobs 2 \
    --o-classification taxa.qza

^C


Now let's have a look at the relative abundances of the different bacterial taxa we have in each sample:

In [ ]:
!qiime taxa barplot \
    --i-table dada2/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization taxa_barplot.qzv

Usage: qiime taxa barplot [OPTIONS]

  This visualizer produces an interactive
  barplot visualization of taxonomies.
  Interactive features include multi-level
  sorting, plot recoloring, sample relabeling,
  and SVG figure export.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                         Feature table to
                         visualize at various
                         taxonomic levels.
                                        [required]
  --i-taxonomy ARTIFACT FeatureData[Taxonomy]
                         Taxonomic annotations
                         for features in the
                         provided feature table.
                         All features in the
                         feature table must have a
                         corresponding taxonomic
                         annotation. Taxonomic
                         annotations that are not
                         present in the feature
                         table will be ignored.
      

We can also collapse data on a particular taxonomic rank using the QIIME 2 [taxa plugin](https://docs.qiime2.org/2021.4/plugins/available/taxa/). Why might we want to look at different taxonomic ranks, rather than just looking at ASVs?

In [ ]:
!qiime taxa collapse \
    --i-table dada2/table.qza \
    --i-taxonomy taxa.qza \
    --p-level 6 \
    --o-collapsed-table genus.qza

Usage: qiime taxa collapse [OPTIONS]

  Collapse groups of features that have the same
  taxonomic assignment through the specified
  level. The frequencies of all features will be
  summed when they are collapsed.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                         Feature table to be
                         collapsed.     [required]
  --i-taxonomy ARTIFACT FeatureData[Taxonomy]
                         Taxonomic annotations
                         for features in the
                         provided feature table.
                         All features in the
                         feature table must have a
                         corresponding taxonomic
                         annotation. Taxonomic
                         annotations that are not
                         present in the feature
                         table will be ignored.
                                        [required]
Parameters:
  --p-level INTEGER      The taxonomic level at

We can export the table and convert it to a .csv file so that we can analyze these data using tools outside of the QIIME 2 environment.

In [ ]:
!qiime tools export \
    --input-path genus.qza \
    --output-path exported
!biom convert -i exported/feature-table.biom -o genus.tsv --to-tsv

Usage: qiime tools export [OPTIONS]

  Exporting extracts (and optionally transforms)
  data stored inside an Artifact or
  Visualization. Note that Visualizations cannot
  be transformed with --output-format

Options:
  --input-path ARTIFACT/VISUALIZATION
                        Path to file that should
                        be exported     [required]
  --output-path PATH    Path to file or directory
                        where data should be
                        exported to     [required]
  --output-format TEXT  Format which the data
                        should be exported as.
                        This option cannot be used
                        with Visualizations
  --help                Show this message and
                        exit.

                    There was a problem with the command:                     
 (1/1) Invalid value for '--input-path': File 'genus.qza' does not exist.
Usage: biom convert [OPTIONS]
Try 'biom convert -h' for help.

Error: Invalid v

Now the data are in a common format and we can use them, for instance, to draw a heatmap using Pandas and Seaborn. No need to understand every bit of code here. This just serves to illustrate that you can get data out of QIIME 2 for custom visualizations.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

abundances = pd.read_table("genus.tsv", skiprows=1, index_col=0)
abundances.index = abundances.index.str.split(";").str[5]       # Use only the genus name
abundances = abundances[~abundances.index.isin(["g__", "__"])]  # remove unclassified genera
abundances = abundances.iloc[0:100]                             # use only the first 100 genera

# Let's do a centered log-ratio transform: log x_i - log mean(x)
transformed = abundances.apply(
    lambda xs: np.log(xs + 0.5) - np.log(xs.mean() + 0.5),
    axis=1)

sns.clustermap(transformed.T, cmap='RdPu', xticklabels=True, figsize=(20, 5))

FileNotFoundError: ignored

In [ ]:
sns.clustermap(transformed.T, cmap='RdPu', xticklabels=True, figsize=(18, 13))
sns_pp = sns.clustermap(transformed.T, cmap='RdPu', xticklabels=True, figsize=(18, 13))
# sns_pp = sns.pairplot(df)
sns_pp.savefig("sns_class_heatmap.png")

# Exercises

Okay, that's enough time in the back seat. 

It's time to take the wheel 🚗 

Now you can dive into the data 🏊

## Exercise 1 - Sample Classification

One pretty basic question we can ask is whether the microbial community composition is predictive of environmental type. 
Could you predict if a sample came from a free-living or host-associated environment from 16S data alone? 

Metadata about the samples is again contained in `metadata.tsv` which looks like this:

In [ ]:
import pandas as pd

pd.read_csv("data/metadata.tsv", sep="\t")

Start with the `classify-samples` action and follow it up by finding and looking at the `heatmap` visualization afterwards that shows important taxa.

We will try to build a machine learning model that can predict whether a microbiome is animal-associated or free-living.

You can also find accuracy results that will tell you how well the phenotype can be predicted from the microbial composition.

I filled in the command for you but it is missing some inputs. Can you complete it? 

Try to use the genus table and not the ASV table here (why?).

**QUESTIONS:**

1) What does it mean for data to be in the 'training' or 'test' set? 

2) How well did this classifier perform? 

3) What ASVs contributed most to model performance? Why do you think these ASVs were so important?

4) Do you think this is a model is broadly useful? Would it perform well on external data that it has not seen yet? Why or why not?


In [ ]:
!rm -rf classifier
!qiime sample-classifier classify-samples \
    --i-table genus.qza \
    --m-metadata-file data/metadata.tsv \
    --m-metadata-column Responses \
    --p-n-jobs 2 \
    --p-test-size 0.33 \
    --p-cv 1 \
    --output-dir classifier 

Okay, we filled in the command here. ASVs are obtained from the data set itself and depend heavily on the specific amplicon (and thus on the primers). To make the model a bit more generalizable, we can collapse counts to the genus level, because you could easily count genera even in data where a different 16S amplicon was used. Ok, let's get to the questions.

**1)** One thing we would like to estimate is how well the model will perform on new data without training it anew. Here "training" means fitting some internal parameters of the model to the data. For instance, fitting coefficients in a linear regression model. What we would like to know now is if the  model still works well on data that was not used for training the model. If it does not, we often call that ["overfitting"](https://en.wikipedia.org/wiki/Overfitting) which means model works well on the training data but horribly on new data. To check for this, we can hold out a few samples as a "test set" -- those will not be used for training but we still know the ground truth for them (i.e. the environment they came from). If we train the model now we can then validate its robustness using the test set to see if it performs well on that data that it hasn't seen before.

**2)** For the performance we can look at the accuracy visualization `classifier/accuracy_results.qzv` by opening it with the [QIIME 2 viewer](https://view.qiime2.org/). This shows the results for the test set. We held out 1/3 of the data for testing which means 5/15 samples. It will show you the real label (animal-associated = yes, free-living = no) and what the model predicted. It should show a near-perfect performance on the test data, meaning that the model as good accuracy in the test set. Yay!

**3)** We might ask which particular genera are actually important to the prediction. A list of up to 50 genera that help the most with classification can be visualized by opening the `classifier/heatmap.qzv` visualization with the QIIME 2 viewer. Each row denotes a group (animal-associated or not) and each column denotes a genus. Each cell is colored by the average abundance of this genus in each group. As you can see for the most part those are mutually exlusive genera. A whole bunch of genera that appear only in free-living microbiota and quite a few that only live in animal-associated microbiota.

**4)** To some degree, we already looked at this question by using our test set. However, even the test set only used data from the 5 environments studied in this course. What would happen if some new kind of environment contained none of the genera that we saw on the heatmap? Our classifier would probably not perform well there. Similarly what would happen if we had assigned taxonomy with SILVA, which would have assigned different genera names? It would probably not perform well there either. So even with a test set the classifier probably *only works on data with similar charateristics* as the training data. 

## Exercise 2 - Plant a Tree

One visualization that we did not spend a lot of time on was the phylogentic tree of our ASVs. Let's change that! 

Let's annotate the tree with our taxonomic classifications and abundances. We will use the empress plugin again, but this time with the `community-plot` option. I filled in a template of the command for you. Can you figure out what has to go in the empty spaces?

**QUESTIONS:**

1) Are some of the branch lengths on the tree longer than you would expect? Do you notice anything interesting or suspicious about the taxonomic identities of these branches?

2) Can you find examples of phyla that are polyphyletic (i.e. where clusters of ASVs from the same phylum are found in different locations on the tree, showing different commmon ancestors)? What about polyphyletic taxa at lower taxonomic levels, like at the family or genus levels? Why do you think these patterns exist?

In [ ]:
!qiime empress community-plot \
    --i-tree tree/rooted_tree.qza \
    --i-feature-table dada2/table.qza \
    --m-sample-metadata-file data/metadata.tsv \
    --m-feature-metadata-file taxa.qza \
    --o-visualization community-tree-viz.qzv

This is what the command looks like with everything filled in. This command will create the same tree as before, but you can now annotate it with a lot more information, and hovering over a node will now also show its taxonomic classification. Another thing that can be helpful is to collapse nodes on a particular taxonomic rank. For instance, you could collapse on the genus level to reduce the large number of external nodes (and the unclassified ones). 

![Example visualization](https://github.com/Gibbons-Lab/isb_course_2021/raw/main/docs/16S/assets/community_tree.png)

Let's get to the questions:

**1)** There are definitely some very long branches, what could be happening? Hovering over those very separate nodes you see that some of them have no classification, sometimes not even on the kingdom level. Interestingly, bacteria are not the only organisms that have a 16S gene. But the GreenGenes database mostly covers bacteria. The ones not classified on the kingdom level are thus likely sequences coming from 16S genes from another Domain, such as archaea. However, many of these very long branches are still classified as bacteria, and in many cases get classified as mitochondria or choloroplasts. First, of all it's really hard to distinguish mitochondria and chloroplast based on the V4 region alone. These organelles have diverged a lot from other bacteria, however they are still related to bacteria, which is somewhat consistent with [endosymbiotic theory](https://en.wikipedia.org/wiki/Symbiogenesis). Mitochondria, chloroplasts, and plastids often still contain their own genomes, seperate from their eukaryotic host cells (i.e. plants, animals, fungi, protists), and their internal 16S genes are still close enough to bacteria that the V4 primers will amplify them.

**2)** In most cases you would expect that taxonomy should match the tree that you obtain by aligning the V4 sequences. After all taxonomies like NCBI, GreenGenes, or SILVA are often built by aligning the the core proteins, full 16S gene, or other parts of the genome. These molecular taxonomies are often designed to be monophyletic. However, initial taxonomic classification of microbes in the 19th and 20th centuries was based upon phenotypic assays. And as we see in the metabolic modeling course, sometimes unrelated taxa can show similar metabolic functionality. This is why you will find, for instance, genera like `Prevotella_1`, `Prevotella_2`, and `Prevotella_3` in SILVA. The sequences from organisms classified originally as Prevotella fell into polyphyletic groups (i.e. they didn't all share the same common ancestor) and were thus split up further into monopyletic clades. When building trees from amplicon sequencing data, you will often see examples of polyphyletic taxa.  You can further explore these polyphyletic taxa by coloring nodes by their environment, and you will often see that many of those polyphyletic groups are found in distinct environments.

<br><br><br>

---

# 🦠 Space for the Project Challenge

You can start working on the project challenge here if you would like. If you rerun all the previous cells in the notebook (`Runtime > Run All`) you should be all set to start working on the challenge. Remember that you can get help in the Slack channel and also in the [Qiime 2 documentation](https://docs.qiime2.org).

In [ ]:
# You can add more code cells with the "+ Code" button on the top right
